In [1]:
%run 1._Add_Psilocybin_to_Coynebacterium_Model.ipynb

No objective coefficients in model. Unclear what should be optimized


In [2]:
model.objective = model.reactions.EX_psi_e # setting the objective towards our product by setting the objective function to its exchange reaction

In [3]:
psilocybin_production = model.optimize().objective_value
print('Maximum theoretical productivity of psilocybin:', psilocybin_production, 'mmol/gDW*h')
print("Theoretical max. yield of psilocybin:", psilocybin_production / (-1*model.reactions.EX_glc_e.flux), "mmol-lyco / mmol-glc")

Maximum theoretical productivity of psilocybin: 5.6929988370778216e-05 mmol/gDW*h
Theoretical max. yield of psilocybin: 1.329527959578845e-05 mmol-lyco / mmol-glc


In [4]:
tryptophan_reactions_c = [] 
for reaction in model.reactions.query("trp_L_c","reaction"): 
    tryptophan_reactions_c.append(reaction.id) # saving all reaction identifiers to the list tryptophan_reactions_c, which are either using or producing L-tryptophan
print("All reactions, which are related to L-tryptophan (c): " ,tryptophan_reactions_c)

All reactions, which are related to L-tryptophan (c):  ['TRPabcpp', 'TRPt2rpp', 'TRPTRS', 'TRPS1', 'TRPS2', 'PROTEIN', 'CrTdc']


In [5]:
tryptophan_reactions_c_Names = []
for reaction in tryptophan_reactions_c: 
    tryptophan_reactions_c_Names.append(model.reactions.get_by_id(reaction).name)
print("All names of reactions, which are related to L-tryptophan (c): " ,tryptophan_reactions_c_Names)

All names of reactions, which are related to L-tryptophan (c):  ['L_trp transport via ABC system ', 'L_tryptophan reversible transport via proton symport ', 'TryptophanyL_tRNA synthetase', 'tryptophan synthase (indoleglycerol phosphate)', 'tryptophan synthase (indole)', 'PROTEIN', '']


In [6]:
# the code below, will knockout all reactions using L-tryptophan (c) and overproduce the synthesis reactions TRPS1 and TRPS2
with model:
    model.objective = model.reactions.EX_psi_e
    model.reactions.TRPt2rpp.bounds = 0, 0
    model.reactions.TRPabcpp.bounds = 0, 0
    model.reactions.get_by_id("PROTEIN").bounds = 0, 0
    model.reactions.TRPS2.lower_bound = model.reactions.TRPS2.upper_bound
    model.reactions.TRPS1.lower_bound = model.reactions.TRPS1.upper_bound
    model.reactions.TRPTRS.bounds = 0, 0
    print('Knocking everything out besides CrTdc reaction and synthesis reactions, which goes towards our product: ', model.optimize().objective_value, 'mmol/gDW*h')


Knocking everything out besides CrTdc reaction and synthesis reactions, which goes towards our product:  5.6929988370778216e-05 mmol/gDW*h


C:\Users\tamwei\Anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


The theoretical maximum yield of psilocybin is still ~5.7 e-05 mmol/gDW*h

In [7]:
#here we knock-out the reaction producing and using psilocin to see whether the flux is looping there
with model:
    model.reactions.psiK2.bounds = 0, 0
    model.reactions.pho.bounds = 0, 0
    model.objective = model.reactions.EX_psi_e
    print(model.optimize().objective_value)

5.692998837082858e-05


The therotical max. yield of psilocybin is still approx 5.7e-05 mmol/gDW*h. Thus the reactions are not influencing psilocybin production 

In [8]:
#The theoretical max yield after creating single knockouts of the reactions is saved to the list production_knock_outs
production_knock_outs = []
for reaction_id in tryptophan_reactions_c:
   with model:
        model.objective = model.reactions.EX_psi_e # setting objective to our product
        model.reactions.get_by_id(reaction_id).bounds = 0, 0 #setting both bounds to 0 --> knock-out
        production_knock_outs.append(model.optimize().objective_value)

production_knock_outs
    


[5.692998837082858e-05,
 5.692998837082858e-05,
 5.692998837082858e-05,
 5.692998837082858e-05,
 5.6929988370828564e-05,
 5.6929988370828564e-05,
 0.0]

In [9]:
#The theoretical max yield, after setting both boundaries to 100  (imitating overproduction) is saved to the list production_overproduction

production_overproduction = []
for reaction_id in tryptophan_reactions_c:
   with model:
        model.objective = model.reactions.EX_psi_e # setting objective to our product
        model.reactions.get_by_id(reaction_id).lower_bound = model.reactions.get_by_id(reaction_id).upper_bound # forcing the model to overexpress the respective reaction
        production_overproduction.append(model.optimize().objective_value)
        print(reaction_id, model.optimize().objective_value)
    

TRPabcpp 0.0
TRPt2rpp 5.6929988370828415e-05
TRPTRS 5.6929988370828415e-05
TRPS1 5.6929988370828415e-05
TRPS2 5.6929988370828415e-05
PROTEIN 3.5236570605778894e-19
CrTdc 5.69299883708285e-05


In [10]:
import pandas as pd # importing pandas to enable work with dataframes

data = {'reaction_id': tryptophan_reactions_c,
        'psilocybin_production [mmol/gDW*h], when knocked out': production_knock_outs,
        'psilocybin_production [mmol/gDW*h], when over-produced': production_overproduction,        
        }

df = pd.DataFrame(data)

df

,reaction_id,"psilocybin_production [mmol/gDW*h], when knocked out","psilocybin_production [mmol/gDW*h], when over-produced"
0,TRPabcpp,0.000057,0.000000e+00
1,TRPt2rpp,0.000057,5.692999e-05
2,TRPTRS,0.000057,5.692999e-05
3,TRPS1,0.000057,5.692999e-05
4,TRPS2,0.000057,5.692999e-05
5,PROTEIN,0.000057,3.523657e-19
6,CrTdc,0.000000,5.692999e-05


None of single-knockouts changed the theoretical max yield of psilocybin. Overproduction was simulated by setting the lower bound of each target (reaction influencing L-tryptophan by either production or usage of L-tryptophan) to the level of the higher bound. With this the model is forced to push flux toward this direction, but still is not resulting in highly unrealistic results, since the boundaries for iCW773 seemed to be realistic (numbers showed exact numbers like e.g. 77.8763742).However the highest theoretical max. yield is not higher than the one calculated before introducing gene modulation. 

Since no manually introduced single knock-out turned out to be a promising strategy, we will check for the amount of exchange reactions, which present in the model iCW773.

In [11]:
print("The model iCW773_psilocybin harbours ",len(model.exchanges), "exchange reactions.")

The model iCW773_psilocybin harbours  163 exchange reactions.


The amount of exchange reaction seems to be quite high with 163 reactions.

In [12]:
# Overexpression of all targets identified within FSEOF
targets = ['ACHMSC', 'G1PACT','PGAMT','SERAT','MTHFR2','GLNS','CYSTL','HSST','METS','ADK1','CYTK1','CYTK2','GK1','NDPK1','RNDR3','CTPS2','GMPS2','CrTdc','psiH','psiK','psiM','psiM2','psitpp','EX_psi_e']
production_overproduction_targets = []
for reaction_id in targets:
   with model:
        model.objective = model.reactions.EX_psi_e # setting objective to our product
        model.reactions.get_by_id(reaction_id).lower_bound = model.reactions.get_by_id(reaction_id).upper_bound # forcing the model to overexpress the respective reaction
        production_overproduction_targets.append(model.optimize().objective_value)
        print(reaction_id, model.optimize().objective_value)


ACHMSC 5.69299883708285e-05
G1PACT 5.69299883708285e-05


C:\Users\tamwei\Anaconda3\lib\site-packages\cobra\util\solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)


PGAMT 5.69299883708285e-05
SERAT 9.754102287778158e-16
MTHFR2 5.69299883708285e-05
GLNS 0.0
CYSTL 0.0
HSST 0.0
METS 0.0
ADK1 5.69299883708285e-05
CYTK1 8.245815711310094e-16
CYTK2 8.245815711310094e-16
GK1 0.0
NDPK1 5.69299883708285e-05
RNDR3 0.0
CTPS2 0.0
GMPS2 0.0
CrTdc 5.692998837081298e-05
psiH 5.692998837081298e-05
psiK 5.692998837081298e-05
psiM 5.692998835127305e-05
psiM2 5.6929988431875245e-05
psitpp 5.692998831818841e-05
EX_psi_e 5.692998831818841e-05


Testing overexpression of identified targets (via FSEOF), does also not result in higher theoretical max yield. The highest achieved yield is ~5.7e-05 and thus not increased to the original yield.